In [99]:
import nltk
import os
import json
import pandas as pd
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import gensim
from gensim import corpora
from gensim.corpora import Dictionary
import numpy as np
fileDir = os.path.dirname(os.path.realpath('__file__'))
print(fileDir)

E:\covid-19-nlp


In [100]:
dirs=["biorxiv_medrxiv"]
docs=[]


for d in dirs:
    for file in os.listdir(f"{d}/{d}/pdf_json"):
        #print(file)
        file_path=f"{d}/{d}/pdf_json/{file}"
        j=json.load(open(file_path,"rb"))
        full_text=""
        abstract=j['abstract']
        title=j['metadata']['title']
        for text in j['body_text']:
            full_text+=text['text']+'\n\n'
        docs.append([title,abstract,full_text])

df=pd.DataFrame(docs,columns=['title','abstract','full_text'])
dfinal = pd.DataFrame(docs,columns=['title','abstract','full_text'])

new_abstract = [] 
for value in df["abstract"]: 
    if (len(value) > 0): 
        new_abstract.append(value[0]['text'])
    else: 
        new_abstract.append("") 
       
df['abstract'] = new_abstract
#df.head()

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text


In [101]:
#Clean data-lower the dataframe, remove stopwords,tokenized text into sentence then into words
xtitleLower = df['title'].to_string(na_rep='').lower()
df['title'] = df['title'].str.lower()
xtitleLower = df['abstract'].to_string(na_rep='').lower()
df['abstract'] = df['abstract'].str.lower()
xtitleLower = df['full_text'].to_string(na_rep='').lower()
df['full_text'] = df['full_text'].str.lower()
stop = stopwords.words('english')
stemmer = SnowballStemmer('english')
df['title'] = df['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['abstract'] = df['abstract'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['full_text'] = df['full_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df["title"] = df['title'].apply(remove_punctuations)
df["abstract"] = df['abstract'].apply(remove_punctuations)
#df["full_text"] = df['full_text'].apply(remove_punctuations)
df['title'] = df['title'].str.replace('\d+', '')
df['abstract'] = df['abstract'].str.replace('\d+', '')
df['full_text'] = df['full_text'].str.replace('\d+', '')
#df['stemmed'] = df['unstemmed'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
#df = df.drop(columns=['unstemmed'])
#print(df.head())
arr = df.to_numpy()


In [102]:
dirs=["biorxiv_medrxiv"]
docs=[]

In [103]:

for d in dirs:
    for file in os.listdir(f"{d}/{d}/pdf_json"):
        #print(file)
        file_path=f"{d}/{d}/pdf_json/{file}"
        j=json.load(open(file_path,"rb"))
        full_text=""
        abstract=j['abstract']
        title=j['metadata']['title']
        for text in j['body_text']:
            full_text+=text['text']+'\n\n'
        docs.append([title,abstract,full_text])

df=pd.DataFrame(docs,columns=['title','abstract','full_text'])
dfinal = pd.DataFrame(docs,columns=['title','abstract','full_text'])

In [104]:
new_abstract = [] 
for value in df["abstract"]: 
    if (len(value) > 0): 
        new_abstract.append(value[0]['text'])
    else: 
        new_abstract.append("") 
       
df['abstract'] = new_abstract
#df.head()

In [105]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [106]:
#tokenization
for i in range(len(arr)):
    for j in range(len(arr[i])):
        arr[i][j] = word_tokenize(arr[i][j])


In [107]:
arrayDictionaries = np.empty(len(arr), dtype=object)
result = []
for i in range(len(arr)):
    result1 = arr[i][0] + arr[i][1] +arr[i][2] 
    arrayDictionaries[i] = np.array(result1)  

In [108]:
#build the dictionary
dictionary = gensim.corpora.Dictionary(arrayDictionaries)

In [109]:
#create corpus 
corpus = [dictionary.doc2bow(arrayDictionary) for arrayDictionary in arrayDictionaries]
tf_idf = gensim.models.TfidfModel(corpus)

In [110]:
# building the index
sims = gensim.similarities.Similarity(fileDir+'/Index',tf_idf[corpus],num_features=len(dictionary))
#print to check that the sims were created
print(sims)
print(type(sims))

Similarity index with 2278 documents in 0 shards (stored under E:\covid-19-nlp/Index)
<class 'gensim.similarities.docsim.Similarity'>


In [111]:
file2_docs = []

with open ('Queries.txt') as f:
    tokens = sent_tokenize(f.read())
    for line in tokens:
        file2_docs.append(line)

print("Number of documents:",len(file2_docs))  
for line in file2_docs:
    query_doc = [w.lower() for w in word_tokenize(line)]
    query_doc_bow = dictionary.doc2bow(query_doc) #update an existing dictionary andcreate bag of words

Number of documents: 1


In [112]:
query_doc_tf_idf = tf_idf[query_doc_bow]
# print(document_number, document_similarity)
print('Comparing Result:', sims[query_doc_tf_idf])

Comparing Result: [0. 0. 0. ... 0. 0. 0.]


In [113]:
sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype=np.float32))
print(sum_of_sims)

3.8611813


In [114]:
#the maximum value
print(np.amax(sims[query_doc_tf_idf]))

0.65469474


In [115]:
#the top 10 similarities
print (sims[query_doc_tf_idf][np.argsort(sims[query_doc_tf_idf])[-10:]])

[0.108917   0.11822368 0.12335537 0.14555898 0.14591417 0.15535428
 0.18939789 0.1907245  0.22733808 0.65469474]


In [116]:
#return the id of the top 10 similarities
print( np.argsort(sims[query_doc_tf_idf])[-10:])

[ 891  835  846 1407  933  864  799  789  615 1112]


In [117]:
#return the tite and the fullbody documents of the top 10 similarities
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])
arrFinal = dfinal.to_numpy()
arrayDocuementsid  = np.argsort(sims[query_doc_tf_idf])[-10:]
for i in range (len(arrayDocuementsid)):
    index = arrayDocuementsid[i]
    print ("The " + ordinal(i+1) + " best search" )
    if(arrFinal[index][0]==''):
        print("Title : not Mentioned" + arrFinal[index][0])
    else:
        print("Title : " + arrFinal[index][0])
        
        
    
    print(arrFinal[index][2])

The 1st best search
Title : not Mentioned
There is an ongoing pandemic of viral pneumonia called Coronavirus Disease 2019 which is caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) infection. 44

At the time of this writing, the disease has been reported to affect 634,835 people in more than 45 200 countries, territories or areas, and cause 29,891 deaths (1). Understanding the clinical and 46 epidemiological characteristics of the disease is important for informing public health decision 47 making, which would enable improvement of surveillance and effective planning of treatment. In in China and in other parts of the world (3-12). However, most of these reports were limited by a 52 small sample size, and the characteristics reported appeared to be inconsistent. For example, in 53 a report involving 99 patients, Chen et al.

(3) noted a much higher proportion of men than 54 women, and suggested that men were generally more susceptible to SARS-CoV-2 infection. On 55


The 2nd best search
Title : Integrative Bioinformatics Analysis Provides Insight into the Molecular Mechanisms of 2019-nCoV
In early December 2019, several cases of pneumonia with unknown causes were reported in Wuhan, Hubei, China (1) . Later, a novel coronavirus, 2019-nCoV was identified based on sequencing of the patient respiratory tract samples. As of February 1, 2020, more than 10,000 confirmed patients and more than 15,000 suspected cases were reported in China, with additional patients being identified in a rapidly growing number internationally. Analyzing of the genome of 2019-nCoV showed that this new virus shared around 80~90% sequence identity to SARS-CoV (2) . Both bioinformatics modeling and in vitro experiments indicated that 2019-nCoV was more likely to use the angiotensin-converting enzyme 2 (ACE2) as the entry receptor (3, 4) . ACE2 is previously known as the receptor for SARS-CoV and HCoV-NL63 (5, 6) . Considering the sequence of 2019-nCoV is similar to the SRAS-CoV


The 3rd best search
Title : No evidence that androgen regulation of pulmonary TMPRSS2 explains sex-discordant COVID-19 outcomes
The December 2019 outbreak caused by the severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) in Wuhan, China, has led to the coronavirus disease 2019 pandemic Zhou et al., 2020) . The highly contagious nature of the disease as well as the lack of vaccines or clinically approved treatments has caused a worldwide public health emergency. Therefore, improved and timely understanding of the human susceptibilities to SARS-CoV-2 will prove invaluable in controlling the pandemic and treatment of those affected.

SARS-CoV-2 enters cells using two host cellular proteins: angiotensin converting enzyme-2 (ACE2) and transmembrane serine protease 2 (TMPRSS2). The virus first employs ACE2 as a cell entry protein, followed by TMPRSS2-mediated proteolytic processing of the SARS-2 spike protein, further facilitating viral entry (Hoffmann et al., 2020) . Targeting the


The 4th best search
Title : Integrated analyses of single-cell atlases reveal age, gender, and smoking status associations with cell type-specific expression of mediators of SARS-CoV-2 viral entry and highlights inflammatory programs in putative target cells
the receptor for both SARS-CoV 40 and SARS-CoV-2. The receptor-binding domain of the SARS-CoV-2 S-protein has a higher binding affinity for human ACE2 than SARS-CoV 36, 41 , whereas the interaction with CD147 (encoded by the gene BSG), another reported receptor for the SARS-CoV-2 S-protein, is weak (CD147: Kd, 0.185 µM vs hACE2: Kd, ~15 nM) 42 . Following receptor binding, the virus gains access to the host cell cytosol through acid-dependent proteolytic cleavage of the S protein. For SARS-CoV, a number of proteases including TMPRSS2 and CTSL cleave at the S1 and S2 boundary and S2 domain (S2') 43 to mediate membrane fusion and virus infectivity. For SARS-CoV-2, both pharmacological inhibition of endogenous TMPRSS2 protein and TMP


The 5th best search
Title : Clinical characteristics of 36 non-survivors with COVID-19 in Wuhan, China
Although the outbreak of Coronavirus disease 2019 has caused over 2200 deaths in China, there was no study about death yet. We aimed to describe the clinical characteristics of non-survivors with COVID-19.

For this retrospective, single-center study, we included 36 non-survivors with COVID-19 in the Fifth Hospital of Wuhan. Cases were confirmed by real-time RT-PCR between Jan 21 and Feb 10, 2020 according to the recommended protocol. The epidemiological, demographic, clinical, laboratory, radiological and treatment data were collected and analyzed. Outcomes were followed up until Feb 14, 2020 . This study was approved by the ethics commissions of the Fifth Hospital of Wuhan, with a waiver of informed consent due to a public health outbreak investigation.

We included 36 patients who died from COVID- 19 author/funder, who has granted medRxiv a license to display the preprint in perpe


The 6th best search
Title : Cigarette smoke triggers the expansion of a subpopulation of respiratory epithelial cells that express the SARS-CoV-2 receptor ACE2
In December 2019, a novel respiratory disease emerged in a seafood market in Wuhan, China 1 . Genomic sequencing demonstrated that the causative agent was a highly-contagious coronavirus, since named SARS-CoV-2 2,3 . The disease, called COVID-19, rapidly spread worldwide, and as of March 2020, more than 600,000 people have been infected and more than 25,000 people have died 4 . No clinically-validated treatment or vaccine for COVID-19 is currently available. Thus, understanding the factors that mediate susceptibility to SARS-CoV-2 is crucial for controlling disease transmission. Molecular analysis has begun to shed light on how SARS-CoV-2 infections occur. Like a related coronavirus that emerged in 2003 5 , SARS-CoV-2 enters human cells by binding to the extracellular domain of Angiotensin Converting Enzyme 2 (ACE2) 3,6 . Impor


The 7th best search
Title : Bulk and single-cell transcriptomics identify tobacco-use disparity in lung gene expression of ACE2, the receptor of 2019-nCov
In the past two decades, pathogenic coronaviruses (CoVs) have caused epidemic infections, including the server acute respiratory syndrome (SARS)-CoV outbreak in 2003, the Middle East Respiratory Syndrome Coronavirus (MERS-CoV) outbreak in 2012 and the current Wuhan 2019 Novel Coronavirus (2019-nCov) outbreak. They typically affect the respiratory tract and cause severe respiratory illnesses. We have learned from SARS-Cov and MERS-Cov that human populations showed disparities in susceptibility to these viruses. For example, epidemiology studies found that males had higher incidence and mortality rates than females. 1, 2 We believe that the susceptibility to the novel 2019-nCov is also different among population groups. In current severe global emergency situation of 2019-nCov outbreak, it is imperative to identify vulnerable and susc


The 8th best search
Title : The relationship of COVID-19 severity with cardiovascular disease and its traditional risk factors: A systematic review and meta-analysis
Cases of coronavirus disease 2019 (COVID-19) are rapidly increasing globally. As of April 5, 2020, more than 1.2 million cases have been confirmed and ~70,000 deaths have been reported in ~180 countries. 1 Several studies have rapidly provided crucial data (e.g., incubation period) related to various aspects of the novel coronavirus (SARS-CoV-2 : severe acute respiratory syndrome coronavirus 2) infection. 2 However, risk factors for the severity and prognosis of COVID-19 are poorly understood. Such information is critical to identify high risk patients and to facilitate planning (e.g., forecasting the need for hospital beds and mechanical ventilators). These risk factors will also have implications for workforce allocation (e.g., assignment of healthcare providers with specific risk factors to positions with reduced risk 


The 9th best search
Title : Prevalence, Severity and Mortality associated with COPD and Smoking in patients with COVID-19: A Rapid Systematic Review and Meta-Analysis Authors contributions
The emergence in Wuhan City, Hubei Province of China of a novel pneumonia of unknown origin on the 31 st of December, 2019 was the start of an outbreak which would later be declared a pandemic by the World Health Organization (WHO) (1). The name COVID-19 (acronym for "coronavirus disease 2019") was coined on the 11 th of February 2020 to describe presentation with severe acute respiratory disease (2). COVID-19 is caused by a novel strain of coronavirus, the severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). This virus belongs to the family of single stranded RNA viruses some of which have been previously described to be responsible for the Severe Acute Respiratory Syndrome (SARS) and Middle East Respiratory Syndrome (MERS) (3, 4) . Although the symptoms and clinical presentation of COVID-


The 10th best search
Title : Smoking is Associated with COVID-19 Progression: A Meta-Analysis
COVID-19, the coronavirus-transmitted infectious disease, has caused a worldwide pandemic. Smoking 1 2 and e-cigarette use 3 increase risk and severity of pulmonary infections because of damage to upper airways and a decrease in pulmonary immune function. In particular, smokers have a higher risk of infection and mortality from Cov-MERS. 4 Two reviews 5 6 of the first 5 papers presenting data on smoking and COVID-19 reached different conclusions. Another review described 6 published case series presenting data on smoking among COVID-19 patients but did not draw a conclusion about the association of severity of COVID-19 with smoking. 7 We reviewed and summarized 12 papers presenting data on the association between smoking and severity of COVID-19.

We conducted a systematic search using PubMed database on April 6, 2020, with the search term: ((smoking) OR (characteristics) OR (risk factors) OR

In [ ]:
from flask import Flask, render_template, request, redirect, url_for

tasks=['transmission, incubation, and environmental stability',
       'COVID-19 risk factors', 
       'virus genetics, origin, and evolution', 
       'non-pharmaceutical interventions',
       'vaccines and therapeutics', 
       'ethical and social science considerations', 
       'diagnostics and surveillance',
       'medical care', 
       'information sharing and inter-sectoral collaboration']
#print(len(tasks))
   
def GetArticle(question):
    question=question
    tokens = sent_tokenize(question)
    for line in tokens:
        file2_docs.append(line)
    for line in file2_docs:
        query_doc = [w.lower() for w in word_tokenize(line)]
        query_doc_bow = dictionary.doc2bow(query_doc) #update an existing dictionary andcreate bag of words
    print("Number of documents:"+str(len(file2_docs)))
    query_doc_tf_idf = tf_idf[query_doc_bow]
    # print(document_number, document_similarity)
    print('Comparing Result:', sims[query_doc_tf_idf])
    sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype=np.float32))
    print(sum_of_sims)
    #the maximum value
    print("Maximum Similiarity")
    print(np.amax(sims[query_doc_tf_idf]))
    #the top 10 similarities
    print (sims[query_doc_tf_idf][np.argsort(sims[query_doc_tf_idf])[-10:]])
    #return the id of the top 10 similarities
    similartiesArray=sims[query_doc_tf_idf][np.argsort(sims[query_doc_tf_idf])[-10:]];
    print( np.argsort(sims[query_doc_tf_idf])[-10:])
    #return the tite and the fullbody documents of the top 10 similarities
    ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])
    arrFinal = dfinal.to_numpy()
    arrayDocuementsid  = np.argsort(sims[query_doc_tf_idf])[-10:]
    Top10=[]
    for i in range (len(arrayDocuementsid)):
        print('Maximum Similiarity of current Document'+str(similartiesArray[i]));
        index = arrayDocuementsid[i]
        print ("The " + ordinal(i+1) + " best search" )
        if(arrFinal[index][0]==''):
            print("Title : not Mentioned" + arrFinal[index][0])
            title="Title not Mentioned" + str(arrFinal[index][0])
        else:
            print("Title : " + arrFinal[index][0])
            title=arrFinal[index][0]
        Top10.append([arrFinal[index][2],title,similartiesArray[i]])
    return Top10
def MultipleQuestions():
    AnswersArray=[]
    for i in range (0,len(tasks)):
        print(i)
        Answer=GetArticle(tasks[i])[9];
        AnswersArray.append(Answer)
    return (AnswersArray);

app = Flask(__name__)
@app.route('/')
def index():
    return render_template("index.html")

@app.route('/', methods=['POST'])
def my_form_post():
    text = request.form['text']
    Answers =GetArticle(text)
    article1,title1,similiarity1=Answers[9]
    article2,title2,similiarity2=Answers[8]
    article3,title3,similiarity3=Answers[7]
    article4,title4,similiarity4=Answers[6]
    article5,title5,similiarity5=Answers[5]
    article6,title6,similiarity6=Answers[4]
    article7,title7,similiarity7=Answers[3]
    article8,title8,similiarity8=Answers[2]
    article9,title9,similiarity9=Answers[1]
    article10,title10,similiarity10=Answers[0]

    return render_template("answer/index.html",Question=text
                           ,article1=article1,title1=title1,similiarity1=similiarity1
                           ,article2=article2,title2=title2,similiarity2=similiarity2
                          ,article3=article3,title3=title3,similiarity3=similiarity3
                          ,article4=article4,title4=title4,similiarity4=similiarity4
                          ,article5=article5,title5=title5,similiarity5=similiarity5
                          ,article7=article7,title7=title7,similiarity7=similiarity7
                          ,article6=article6,title6=title6,similiarity6=similiarity6
                          ,article8=article8,title8=title8,similiarity8=similiarity8
                          ,article9=article9,title9=title9,similiarity9=similiarity9
                           ,article10=article10,title10=title10,similiarity10=similiarity10
                          
                          
                          )

@app.route('/faq')
def faq():
    Answers =MultipleQuestions(); 
    article1,title1,similiarity1=Answers[0]
    article2,title2,similiarity2=Answers[1]
    article3,title3,similiarity3=Answers[2]
    article4,title4,similiarity4=Answers[3]
    article5,title5,similiarity5=Answers[4]
    article6,title6,similiarity6=Answers[5]
    article7,title7,similiarity7=Answers[6]
    article8,title8,similiarity8=Answers[7]
    article9,title9,similiarity9=Answers[8]

    print("FAQ HERE")
    #print(similiarity1)
    return render_template("faq/index.html",
                           article1=article1,title1=title1
                           ,article2=article2,title2=title2
                          ,article3=article3,title3=title3
                          ,article4=article4,title4=title4
                          ,article5=article5,title5=title5
                          ,article7=article7,title7=title7
                          ,article6=article6,title6=title6
                          ,article8=article8,title8=title8
                          ,article9=article9,title9=title9)
 

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/May/2020 01:32:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2020 01:32:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2020 01:32:24] "GET / HTTP/1.1" 200 -


Number of documents:2
Comparing Result: [0.         0.         0.         ... 0.00148189 0.         0.        ]
4.6586294
Maximum Similiarity
0.1647856
[0.05636115 0.05703232 0.05949826 0.06513985 0.09237453 0.11066813
 0.12400745 0.12550147 0.1512416  0.1647856 ]
[1266  623 2254 1773  708 1048  931 1337 1041  644]
Maximum Similiarity of current Document0.05636115
The 1st best search
Title : Epidemiological and ecological modelling reveal diversity in upper respiratory tract microbial population structures from a cross-sectional community swabbing study Key words Ecology, Epidemiology, Respiratory Infection Running Title Application of epidemiological and ecological modelling to microbial populations Corresponding Author
Maximum Similiarity of current Document0.057032317
The 2nd best search
Title : An Effective CTL Peptide Vaccine for Ebola Zaire Based on Survivors' CD8+ Targeting of a Particular Nucleocapsid Protein Epitope with Potential Implications for COVID-19 Vaccine Design
Maxim

127.0.0.1 - - [14/May/2020 01:32:26] "POST / HTTP/1.1" 200 -


0
Number of documents:3
Comparing Result: [0.         0.00239954 0.00657212 ... 0.00462065 0.00140222 0.        ]
16.598541
Maximum Similiarity
0.31352836
[0.11928368 0.1196094  0.12171915 0.13552599 0.13590983 0.1370548
 0.15111834 0.16413192 0.17724138 0.31352836]
[2253  265 1368 1824  978   52  160 2144 1696 2092]
Maximum Similiarity of current Document0.119283676
The 1st best search
Title : Transmission of corona virus disease 2019 during the incubation period may lead to a quarantine loophole
Maximum Similiarity of current Document0.1196094
The 2nd best search
Title : not Mentioned
Maximum Similiarity of current Document0.12171915
The 3rd best search
Title : Title: Is the spread of COVID-19 across countries influenced by environmental, economic and social factors?
Maximum Similiarity of current Document0.13552599
The 4th best search
Title : Is a 14-day quarantine period optimal for effectively controlling coronavirus disease 2019 (COVID-19)?
Maximum Similiarity of current Document

Title : If long-term suppression is not possible, how do we minimize mortality for COVID-19 and other emerging infectious disease outbreaks?
4
Number of documents:7
Comparing Result: [0.         0.00186851 0.         ... 0.00599675 0.         0.        ]
6.157095
Maximum Similiarity
0.10613852
[0.06528234 0.06807995 0.0707794  0.07773875 0.0802798  0.08870413
 0.09033341 0.09592824 0.10429777 0.10613852]
[ 708 1903 1337 1855  353  492 1041  644  931  538]
Maximum Similiarity of current Document0.06528234
The 1st best search
Title : Shotgun proteomics of SARS-CoV-2 infected cells and its application to the optimisation of whole viral particle antigen production for vaccines
Maximum Similiarity of current Document0.06807995
The 2nd best search
Title : COMPUTATIONAL PREDICTION OF THE COMPREHENSIVE SARS-COV-2 VS. HUMAN INTERACTOME TO GUIDE THE DESIGN OF THERAPEUTICS A PREPRINT -NOT PEER-REVIEWED
Maximum Similiarity of current Document0.0707794
The 3rd best search
Title : Fractional Dosing 

Title : Epidemiological, Clinical Characteristics and Outcome of Medical Staff Infected with COVID-19 in Wuhan, China: A Retrospective Case Series Analysis
Maximum Similiarity of current Document0.229061
The 10th best search
Title : Flattening the curve before it flattens us: hospital critical care capacity limits and mortality from novel coronavirus (SARS-CoV2) cases in US counties
8
Number of documents:11
Comparing Result: [0.         0.00505337 0.00230032 ... 0.00383356 0.         0.        ]
5.5600777
Maximum Similiarity
0.22963426
[0.04037378 0.04175462 0.0477051  0.04829083 0.04935154 0.0546926
 0.07486511 0.08427481 0.14867748 0.22963426]
[ 731  903  724 1276  577  820  131  311 1270  665]
Maximum Similiarity of current Document0.040373784
The 1st best search
Title : Differential ventilation using flow control valves as a potential bridge to full ventilatory support during the COVID-19 crisis: From bench to bedside
Maximum Similiarity of current Document0.04175462
The 2nd best s

127.0.0.1 - - [14/May/2020 01:32:33] "GET /faq HTTP/1.1" 200 -
